In [6]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import load_wine
from sklearn.tree import DecisionTreeClassifier
from sklearn import model_selection

In [8]:
X, y = load_wine(return_X_y=True)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25)

In [10]:
def get_error(pred, y):
    return np.sum((pred != y).astype(int)) / len(y)

In [11]:
def adaboost(X, y, N):

    n_objects = len(X)

    n_classes = len(np.unique((y)))

    w = np.ones(n_objects) / n_objects

    models = []

    for n in range(N):
        clf = DecisionTreeClassifier(max_depth=1)
        clf.fit(X, y, w)

        predictions = clf.predict(X)
        e = get_error(predictions, y)
        if e >= 1 - 1/n_classes: 
            break

        alpha = 0.5 * np.log((1 - e) / e)

        match = predictions == y

        w[np.logical_not(match)] *= np.exp(alpha)
        w[match] *= np.exp(-alpha)

        w /= w.sum()

        models.append((alpha, clf))
    
    return models

In [12]:
def predict(X, models):
    
    n_classes = 2
    n_objects = len(X)
    
    y_pred = np.zeros((n_objects, n_classes))
    
    for alpha, clf in models:
        prediction = clf.predict(X)
        y_pred[range(n_objects), prediction] += alpha
    
    y_pred = np.argmax(y_pred, axis=1)
    
    return y_pred

In [13]:
N = 50

models = adaboost(X_train, y_train, N)

In [15]:
# Для начала отмасштабируем выборку
X_ = X_train.astype(float)

rows, cols = X_.shape

# центрирование - вычитание из каждого значения среднего по строке
means = X_.mean(0)
for i in range(rows):
    for j in range(cols):
        X_[i, j] -= means[j]

# деление каждого значения на стандартное отклонение
std = np.std(X_, axis=0)
for i in range(cols):
    for j in range(rows):
        X_[j][i] /= std[i]

In [16]:
X.shape

(178, 13)

In [17]:
# Найдем собственные векторы и собственные значения
 
covariance_matrix = X_.T.dot(X_)

eig_values, eig_vectors = np.linalg.eig(covariance_matrix)

# сформируем список кортежей (собственное значение, собственный вектор)
eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:, i]) for i in range(len(eig_values))]

# и отсортируем список по убыванию собственных значений
eig_pairs.sort(key=lambda x: x[0], reverse=True)

# Сформируем вектор весов из собственных векторов, соответствующих первым двум главным компонентам
W = np.hstack((eig_pairs[0][1].reshape(13,1), eig_pairs[1][1].reshape(13,1)))

print(f'Матрица весов W:\n', W)

Матрица весов W:
 [[ 0.13524049 -0.50019318]
 [-0.25847261 -0.21668342]
 [-0.04458299 -0.29581374]
 [-0.28866498  0.05630228]
 [ 0.13700106 -0.28427631]
 [ 0.3773666  -0.07196732]
 [ 0.4105697   0.00703391]
 [-0.2878779   0.03003469]
 [ 0.3143246  -0.06524728]
 [-0.0992929  -0.52005568]
 [ 0.30528166  0.31270164]
 [ 0.37348704  0.15650882]
 [ 0.2749951  -0.35811037]]


In [18]:
new_X_train=X_.dot(W)

In [19]:
new_X_test=X_test[:, :]

In [20]:
# центрирование - вычитание из каждого значения среднего по строке
for j in range(cols):
    new_X_test[:, j] -= means[j]

# деление каждого значения на стандартное отклонение
for i in range(cols):
    new_X_test[:, i] /= std[i]

In [21]:
new_X_test=new_X_test.dot(W)

In [25]:
def accuracy(pred, y):
    return (sum(pred == y) / len(y))

In [23]:
N = 50 

models = adaboost(new_X_train, y_train, N)

In [27]:
print(f'Точность алгоритма на обучающей выборке: {(1 - get_error(predict(new_X_train, models), y_train)) * 100:.3f}')

IndexError: index 2 is out of bounds for axis 1 with size 2

In [28]:
print(f'Точность алгоритма на тестовой выборке: {(1 - get_error(predict(new_X_test, models), y_test)) * 100:.3f}')

IndexError: index 2 is out of bounds for axis 1 with size 2